#sqllite  와 데이터 프레임 실습 카피본

In [11]:
#import library 
import sqlite3 
import pandas as pd
import os

In [2]:
#colab 에 드라이브 연결 구글 드라이브 마운트 
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#리눅스 명령어 
!pwd

/content


In [5]:
#작업 폴더로 이동 
os.chdir("/content/drive/My Drive/인공지능사관학교/빅데이터/copy")

In [6]:
!pwd

/content/drive/My Drive/인공지능사관학교/빅데이터/copy


In [7]:
!ls

ex_0101.ipynb  ex_0102.ipynb  sql1.ipynb  sql2.ipynb


#데이터 프레임의 내용을 데이터 베이스로 옮겨 본다.

In [9]:
os.chdir("../data")
df = pd.read_csv("data_studentlist_en.csv",header="infer")
#header = "infer" 는 첫행을 열이름으로 쓸건지 결정하는 매개변수 


In [10]:
df

,name,gender,age,grade,absence,bloodtype,height,weight
0,Jared Diamond,M,23,3,Y,O,165.3,68.2
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
2,Brian Martin,M,24,4,N,B,175.0,80.1
3,David Hassel,M,23,3,N,AB,182.1,85.7
4,Clara Rodriquez,F,20,1,Y,A,168.0,49.5
5,Jennifer Lorentz,F,21,2,N,O,162.0,52.0
6,Susan Clark,F,22,1,N,O,155.2,45.3
7,Margareth Jones,F,23,1,N,A,176.9,55.0
8,John Bertsch,M,23,3,N,B,178.5,64.2
9,Jake Timmerman,M,22,2,N,B,176.1,61.3


In [13]:
print("shape:{}".format(df.shape),"ndim".format(df.ndim))
print(df.info())

shape:(17, 8) ndim
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       17 non-null     object 
 1   gender     17 non-null     object 
 2   age        17 non-null     int64  
 3   grade      17 non-null     int64  
 4   absence    17 non-null     object 
 5   bloodtype  17 non-null     object 
 6   height     17 non-null     float64
 7   weight     17 non-null     float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ KB
None


In [ ]:
#행 데이터 
nrows = df.shape[0]
nrows #행 데이터 17 저장

my_header = df.columns
my_header #columns 의 이름 데이터 저장 

df.head(3)

## 테이블 내용 가져오기 

In [20]:
#SQLITE 의 데이터 베이스 생성 + cursor 연결 
conn = sqlite3.connect("student.db")
cur = conn.cursor()

In [21]:
#sql 테이블 생성
sql = "CREATE TABLE IF NOT EXISTS studentlist (name TEXT,gender TEXT,age INT ,grade INT,absence TEXT,bloodtype TEXT,height REAL,weight REAL);"
cur.execute(sql)
conn.commit() #sql 문중에 select 를 제외 하고는 테이블 변경이나 생성 삭제 업데이트는 무조건 commit 을 해야한다


In [22]:
#sql 테이블 내용 삭제 
sql = "DELETE FROM studentlist;"
cur.execute(sql)
res = cur.fetchall()
print(res) #delete 연산을 수행 했기 때문에 아무것도 안뜬다 


[]


In [23]:
df.iloc[0].values #iloc 로 가져오면 넘파이 배열로 하나씩 가져온다.

array(['Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2], dtype=object)

In [25]:
#studentlist 테이블 행 추가 
sql = "INSERT INTO studentlist VALUES(?, ?, ?, ?, ?, ?, ?, ? );"
for i in range(nrows):
  x = df.iloc[i].values #행데이터를 가져오고 싶을때는 df.iloc[행 인덱스].values  = 값을 가져온다.
  x = list(x)
  x[2] = int(x[2])
  x[3] = int(x[3])

  cur.execute(sql,x)
  conn.commit()



['Andrew Daley', 'M', 21, 1, 'N', 'A', 169.2, 62.2]


In [26]:
#테이블 내용 보기 
sql = "SELECT * FROM studentlist;"
cur.execute(sql)
res = cur.fetchall()
for x in res:
  print(x)

('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2)
("Sarah O'Donnel", 'F', 22, 2, 'N', 'AB', 170.1, 53.0)
('Brian Martin', 'M', 24, 4, 'N', 'B', 175.0, 80.1)
('David Hassel', 'M', 23, 3, 'N', 'AB', 182.1, 85.7)
('Clara Rodriquez', 'F', 20, 1, 'Y', 'A', 168.0, 49.5)
('Jennifer Lorentz', 'F', 21, 2, 'N', 'O', 162.0, 52.0)
('Susan Clark', 'F', 22, 1, 'N', 'O', 155.2, 45.3)
('Margareth Jones', 'F', 23, 1, 'N', 'A', 176.9, 55.0)
('John Bertsch', 'M', 23, 3, 'N', 'B', 178.5, 64.2)
('Jake Timmerman', 'M', 22, 2, 'N', 'B', 176.1, 61.3)
('Joshua Connor', 'M', 24, 4, 'Y', 'O', 167.1, 62.0)
('John Matsuda', 'M', 22, 2, 'N', 'AB', 180.0, 75.8)
('Eddy Johnson', 'M', 21, 1, 'N', 'A', 162.2, 55.3)
('Rebecah Anderson', 'F', 23, 3, 'N', 'O', 176.1, 53.1)
('Linda Carter', 'F', 22, 2, 'N', 'B', 158.2, 45.2)
('Richard Swayze', 'M', 24, 4, 'Y', 'B', 168.6, 70.2)
('Andrew Daley', 'M', 21, 1, 'N', 'A', 169.2, 62.2)
('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2)
("Sarah O'Donnel", 'F', 22, 2, 'N', 'AB

In [27]:
res # 전체 행은 리스트 형태 한행은 튜플 형태로 저장되어있다.

[('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2),
 ("Sarah O'Donnel", 'F', 22, 2, 'N', 'AB', 170.1, 53.0),
 ('Brian Martin', 'M', 24, 4, 'N', 'B', 175.0, 80.1),
 ('David Hassel', 'M', 23, 3, 'N', 'AB', 182.1, 85.7),
 ('Clara Rodriquez', 'F', 20, 1, 'Y', 'A', 168.0, 49.5),
 ('Jennifer Lorentz', 'F', 21, 2, 'N', 'O', 162.0, 52.0),
 ('Susan Clark', 'F', 22, 1, 'N', 'O', 155.2, 45.3),
 ('Margareth Jones', 'F', 23, 1, 'N', 'A', 176.9, 55.0),
 ('John Bertsch', 'M', 23, 3, 'N', 'B', 178.5, 64.2),
 ('Jake Timmerman', 'M', 22, 2, 'N', 'B', 176.1, 61.3),
 ('Joshua Connor', 'M', 24, 4, 'Y', 'O', 167.1, 62.0),
 ('John Matsuda', 'M', 22, 2, 'N', 'AB', 180.0, 75.8),
 ('Eddy Johnson', 'M', 21, 1, 'N', 'A', 162.2, 55.3),
 ('Rebecah Anderson', 'F', 23, 3, 'N', 'O', 176.1, 53.1),
 ('Linda Carter', 'F', 22, 2, 'N', 'B', 158.2, 45.2),
 ('Richard Swayze', 'M', 24, 4, 'Y', 'B', 168.6, 70.2),
 ('Andrew Daley', 'M', 21, 1, 'N', 'A', 169.2, 62.2),
 ('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2),
 ("

In [29]:
#새로운 데이터 프레임 생성  & 확인
df_new  = pd.DataFrame(data = res,columns = my_header , index = None)
df_new.head(5)

,name,gender,age,grade,absence,bloodtype,height,weight
0,Jared Diamond,M,23,3,Y,O,165.3,68.2
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
2,Brian Martin,M,24,4,N,B,175.0,80.1
3,David Hassel,M,23,3,N,AB,182.1,85.7
4,Clara Rodriquez,F,20,1,Y,A,168.0,49.5


In [30]:
#테이블 삭제 
sql = "DROP TABLE studentlist;"
cur.execute(sql)
conn.commit()

conn.close()